In [1]:
import numpy as np
import xarray as xr

import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
import glob
import pytz
from scipy.signal import welch, csd
from scipy.stats import chi2
from process_fast_data.fast_data_calculate_spectra_nomrd import calculate_mrd_for_df, fast_data_files_to_dataframe

# Open SOS slow data

In [2]:
sos_df = pd.read_parquet("../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags9000_pf10.parquet")
sos_df_localtime = utils.modify_df_timezone( sos_df, 'UTC', 'US/Mountain')

## Plot wind directions for case studies

In [3]:
def wind_chart(data):
    return alt.Chart(data).mark_line().encode(        
        alt.X('time:T'),
        alt.Y('value:Q'),
        alt.Color('height:O').scale(scheme='rainbow')
    ).properties(height=150)
def vertical_lines_chart(data):
    return alt.Chart(
        data
    ).mark_rule(color='red', strokeDash=[4, 2]).encode(
        alt.X('time:T')
    )


sos_dir_df = sos_df_localtime[sos_df_localtime.variable.isin([
        'dir_3m_c', 'dir_10m_c', 'dir_20m_c'
    ])].set_index('time')

(
    (wind_chart(sos_dir_df.loc['20230409'].reset_index())   + vertical_lines_chart(pd.DataFrame({'time': ['2023-04-09T15:00:00', '2023-04-09T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20230418'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2023-04-18T15:00:00', '2023-04-18T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20221221'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2022-12-21T15:00:00', '2022-12-21T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20230304'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2023-03-04T15:00:00', '2023-03-04T18:00:00']})))
)

alt.HConcatChart(...)

# Open SOS fast data

In [6]:
apr18_df = fast_data_files_to_dataframe([
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_21.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_22.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_23.nc',
],  rotation='double')

apr18_df = utils.modify_df_timezone(apr18_df, pytz.UTC, "US/Mountain")

mean u, v, w after first rotation  u_2m_c    1.723918e+00
v_2m_c    1.039858e-07
w_2m_c   -1.881205e-03
dtype: float32
mean u, v, w after first rotation  u_3m_c    1.840870e+00
v_3m_c   -9.946470e-08
w_3m_c   -1.804663e-02
dtype: float32
mean u, v, w after first rotation  u_5m_c    1.946010e+00
v_5m_c    1.785843e-07
w_5m_c   -4.764963e-02
dtype: float32
mean u, v, w after first rotation  u_10m_c    1.988550e+00
v_10m_c   -2.509223e-07
w_10m_c   -1.132471e-01
dtype: float32
mean u, v, w after first rotation  u_15m_c    1.903841e+00
v_15m_c    5.425347e-08
w_15m_c   -1.553703e-01
dtype: float32
mean u, v, w after first rotation  u_20m_c    1.892757e+00
v_20m_c   -4.973235e-08
w_20m_c   -2.244760e-01
dtype: float32
mean u, v, w after first rotation  u_3m_uw    1.895535e+00
v_3m_uw    2.034505e-07
w_3m_uw    2.985147e-02
dtype: float32
mean u, v, w after first rotation  u_10m_uw    2.055680e+00
v_10m_uw   -1.876266e-07
w_10m_uw   -6.043224e-02
dtype: float32
mean u, v, w after first rotat

In [8]:
apr18_u_w_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
apr18_u_w_3m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_d', 'w_3m_d', shift = 2000, parallelism=8)
apr18_u_w_3m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_ue', 'w_3m_ue', shift = 2000, parallelism=8)
apr18_u_w_3m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_uw', 'w_3m_uw', shift = 2000, parallelism=8)

Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:15<00:00,  2.68it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.83it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.89it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.92it/s]


In [9]:
apr18_u_u_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'u_3m_c', shift = 2000, parallelism=8)
apr18_u_u_3m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_d', 'u_3m_d', shift = 2000, parallelism=8)
apr18_u_u_3m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_ue', 'u_3m_ue', shift = 2000, parallelism=8)
apr18_u_u_3m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_uw', 'u_3m_uw', shift = 2000, parallelism=8)

apr18_w_w_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
apr18_w_w_3m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_d', 'w_3m_d', shift = 2000, parallelism=8)
apr18_w_w_3m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_ue', 'w_3m_ue', shift = 2000, parallelism=8)
apr18_w_w_3m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_uw', 'w_3m_uw', shift = 2000, parallelism=8)

Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.02it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.00it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


...

100%|██████████| 42/42 [00:13<00:00,  3.02it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.03it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.02it/s]


In [11]:
apr18_u_w_10m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_c', 'w_10m_c', shift = 2000, parallelism=8)
apr18_u_w_10m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_d', 'w_10m_d', shift = 2000, parallelism=8)
apr18_u_w_10m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_ue', 'w_10m_ue', shift = 2000, parallelism=8)
apr18_u_w_10m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_uw', 'w_10m_uw', shift = 2000, parallelism=8)

Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:15<00:00,  2.66it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.01it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.83it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.93it/s]


In [ ]:
base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log').title('tau (s)'),
)
def mrd_plot(data):
    return alt.layer(
        base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q')),
        base.mark_line().encode(alt.Y('mean(Co):Q')),
        data=data
    ).properties(width=200, height=200)

In [ ]:
(mrd_plot(apr18_u_w_3m_c_mrd_df) | mrd_plot(apr18_u_w_3m_d_mrd_df) | mrd_plot(apr18_u_w_3m_ue_mrd_df) | mrd_plot(apr18_u_w_3m_uw_mrd_df)).resolve_scale(y='shared')

In [ ]:
(mrd_plot(apr18_u_u_3m_c_mrd_df) | mrd_plot(apr18_u_u_3m_d_mrd_df) | mrd_plot(apr18_u_u_3m_ue_mrd_df) | mrd_plot(apr18_u_u_3m_uw_mrd_df)).resolve_scale(y='shared')

In [ ]:
(mrd_plot(apr18_w_w_3m_c_mrd_df) | mrd_plot(apr18_w_w_3m_d_mrd_df) | mrd_plot(apr18_w_w_3m_ue_mrd_df) | mrd_plot(apr18_w_w_3m_uw_mrd_df)).resolve_scale(y='shared')

In [ ]:
(mrd_plot(apr18_u_w_10m_c_mrd_df) | mrd_plot(apr18_u_w_10m_d_mrd_df) | mrd_plot(apr18_u_w_10m_ue_mrd_df) | mrd_plot(apr18_u_w_10m_uw_mrd_df)).resolve_scale(y='shared')

In [ ]:
apr18_u_w_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
apr18_v_w_mrd_df = calculate_mrd_for_df(apr18_df, 'v_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
apr18_u_v_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'v_3m_c', shift = 2000, parallelism=8)

In [ ]:
apr18_u_u_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'u_3m_c', shift = 2000, parallelism=8)
apr18_v_v_mrd_df = calculate_mrd_for_df(apr18_df, 'v_3m_c', 'v_3m_c', shift = 2000, parallelism=8)
apr18_w_w_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_c', 'w_3m_c', shift = 2000, parallelism=8)

In [ ]:
apr18_u_w_mrd_df
apr18_v_w_mrd_df
apr18_u_v_mrd_df
apr18_u_u_mrd_df
apr18_v_v_mrd_df
apr18_w_w_mrd_df

In [ ]:
mrd_uw_sail
mrd_vw_sail
mrd_uv_sail
mrd_uu_sail
mrd_vv_sail
mrd_ww_sail

In [ ]:
(
    (
        mrd_plot(mrd_uu_sail).properties(title="u'u' at GOT") | 
        mrd_plot(mrd_vv_sail).properties(title="v'v' at GOT")
    ).resolve_scale(x='shared', y='shared') & (
        mrd_plot(apr18_u_u_mrd_df).properties(title="u'u' at KPS") | 
        mrd_plot(apr18_v_v_mrd_df).properties(title="v'v' at KPS")
).resolve_scale(x='shared', y='shared')
).resolve_scale(x='shared', y='shared') | (
    mrd_plot(mrd_ww_sail).properties(title="w'w' at GOT") &
    mrd_plot(apr18_w_w_mrd_df).properties(title="w'w' at KPS")
).resolve_scale(x='shared', y='shared')

In [ ]:
(
    (
        mrd_plot(mrd_uw_sail).properties(title="u'w' at GOT") | 
        mrd_plot(mrd_vw_sail).properties(title="v'w' at GOT")
    ).resolve_scale(x='shared', y='shared') & (
        mrd_plot(apr18_u_w_mrd_df).properties(title="u'w' at KPS") | 
        mrd_plot(apr18_v_w_mrd_df).properties(title="v'w' at KPS")
).resolve_scale(x='shared', y='shared')
).resolve_scale(x='shared', y='shared') | (
    mrd_plot(mrd_uv_sail).properties(title="u'v' at GOT") &
    mrd_plot(apr18_u_v_mrd_df).properties(title="u'v' at KPS")
).resolve_scale(x='shared', y='shared')

In [ ]:
dec21_df = fast_data_files_to_dataframe([
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_21.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_22.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_23.nc',
])
dec21_df['u_3m_c'] = dec21_df['u_3m_c'].ffill()
dec21_df['v_3m_c'] = dec21_df['v_3m_c'].ffill()
dec21_df['w_3m_c'] = dec21_df['w_3m_c'].ffill()
dec21_u_w_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
dec21_v_w_mrd_df = calculate_mrd_for_df(dec21_df, 'v_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
dec21_u_v_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_c', 'v_3m_c', shift = 2000, parallelism=8)

In [ ]:
((
    (
        mrd_plot(dec21_u_w_mrd_df).properties(height=200, width=200, title="u'w' at KPS, Dec 21") | 
        mrd_plot(dec21_v_w_mrd_df).properties(height=200, width=200, title="v'w' at KPS, Dec 21")
    ).resolve_scale(x='shared', y='shared') & (
        mrd_plot(apr18_u_w_mrd_df).properties(height=200, width=200, title="u'w' at KPS, Apr 18") | 
        mrd_plot(apr18_v_w_mrd_df).properties(height=200, width=200, title="v'w' at KPS, Apr 18")
).resolve_scale(x='shared', y='shared')
).resolve_scale(x='shared', y='shared') | (
    mrd_plot(dec21_u_v_mrd_df).properties(height=200, width=200, title="u'v' at KPS, Dec 21") &
    mrd_plot(apr18_u_v_mrd_df).properties(height=200, width=200, title="u'v' at KPS, Apr 18")
).resolve_scale(x='shared', y='shared')).display(renderer='svg')

In [ ]:
# '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_20230419_02.nc',

apr18_1000_1200_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_16.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_17.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)
apr18_1200_1400_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_18.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_19.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)
apr18_1400_1600_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_20.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_21.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)
apr18_1600_1800_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_22.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_23.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)
apr18_1800_2000_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_00.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_01.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)
apr18_2000_2200_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_02.nc',
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_03.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c',  'w_3m_c', shift = 1000, parallelism=8
)


In [ ]:
mrd_df = pd.concat([
    apr18_1000_1200_u_w_mrd_df.assign(hour = '1000_1200'),
    apr18_1200_1400_u_w_mrd_df.assign(hour = '1200_1400'),
    apr18_1400_1600_u_w_mrd_df.assign(hour = '1400_1600'),
    apr18_1600_1800_u_w_mrd_df.assign(hour = '1600_1800'),
    apr18_1800_2000_u_w_mrd_df.assign(hour = '1800_2000'),
    apr18_2000_2200_u_w_mrd_df.assign(hour = '2000_2200'),
])
mrd_df['average_time'] = mrd_df.start_time + 0.5*(mrd_df.end_time - mrd_df.start_time)

(alt.Chart(mrd_df).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q'),
    alt.Color('hour:N')
) | alt.Chart(mrd_df.query("hour == '1600_1800'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q'),
    alt.Color('hoursminutes(average_time):O').scale(scheme='purpleorange')
)).resolve_scale(color='independent')

In [ ]:
# '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_20230419_02.nc',

apr18_2000_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_02.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c', 
    'w_3m_c', 
    shift = 200,
    parallelism=8
)
apr18_2100_u_w_mrd_df = calculate_mrd_for_df(
    fast_data_files_to_dataframe([
        '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230419_03.nc',
    ]).set_index('time')[['u_3m_c', 'v_3m_c', 'w_3m_c']].ffill().reset_index(), 
    'u_3m_c', 
    'w_3m_c', 
    shift = 200,
    parallelism=8
)



In [ ]:
mrd_df = pd.concat([
    apr18_2000_u_w_mrd_df.assign(hour = 20),
    apr18_2100_u_w_mrd_df.assign(hour = 21),
])

alt.Chart(mrd_df).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q'),
    alt.Color('iteration:Q'),
    alt.Facet('hour:N')
)

# Load preprocessed spectra, compare across towers

In [ ]:
spectra_mar04 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-03-04 14:00:00_2023-03-04 16:59:59.950000.parquet")
spectra_mar04['height'] = spectra_mar04['loc'].str.split('m').str[0].astype('int')
spectra_mar04['tower'] = spectra_mar04['loc'].str.split('m_').str[1]
spectra_mar04.covariance.unique()

In [ ]:
(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'w_tc_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'w_h2o_'").query("height == 3")
).facet('height') )

In [ ]:
(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'u_u_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'v_v_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'w_w_'").query("height == 3")
).facet('height'))

In [ ]:
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'u_w_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'v_w_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'u_v_'").query("height == 3")
).facet('height')

In [ ]:
spectra_apr18 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-18 15:00:00_2023-04-18 17:59:59.950000.parquet")
spectra_apr18['height'] = spectra_apr18['loc'].str.split('m').str[0].astype('int')
spectra_apr18['tower'] = spectra_apr18['loc'].str.split('m_').str[1]
spectra_apr18.covariance.unique()

base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Color('tower'),
).properties(width=200, height=150)

(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 18') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_1221.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Dec 21') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr09.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 09')).resolve_scale(y='shared', x='shared')

In [ ]:
apr18_df = prep_fast_data([
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_21.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_22.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_23.nc',
])

In [ ]:
len(apr18_df.set_index('time')['u_3m_c']), len(apr18_df.set_index('time')['u_3m_c'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_d']), len(apr18_df.set_index('time')['u_3m_d'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_uw']), len(apr18_df.set_index('time')['u_3m_uw'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_ue']), len(apr18_df.set_index('time')['u_3m_ue'].dropna())

In [ ]:
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 18, 1500-1800')

In [ ]:
spectra_1221 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2022-12-21 14:00:00_2022-12-21 16:59:59.950000.parquet")
spectra_1221['height'] = spectra_1221['loc'].str.split('m').str[0].astype('int')
spectra_1221['tower'] = spectra_1221['loc'].str.split('m_').str[1]

spectra_apr18 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-18 15:00:00_2023-04-18 17:59:59.950000_s1000.parquet")
spectra_apr18['height'] = spectra_apr18['loc'].str.split('m').str[0].astype('int')
spectra_apr18['tower'] = spectra_apr18['loc'].str.split('m_').str[1]

spectra_apr09 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-09 15:00:00_2023-04-09 17:59:59.950000.parquet")
spectra_apr09['height'] = spectra_apr09['loc'].str.split('m').str[0].astype('int')
spectra_apr09['tower'] = spectra_apr09['loc'].str.split('m_').str[1]

base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Color('tower'),
).properties(width=200, height=150)

(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 10")
).facet('height').properties(title='Apr 18') |
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr09.query("covariance == 'u_w_'").query("height == 10")
).facet('height').properties(title='Apr 09')).resolve_scale(x='shared')

In [ ]:
alt.Chart(
    sos_df_localtime.query("measurement == 'wind speed'").query("tower == 'c'").set_index('time').sort_index().loc['20230409 1200': '20230409 1500'].reset_index()
).mark_line().encode(
    alt.X('value:Q'),
    alt.Y('height:Q'),
    alt.Facet('time:T'),
    order='height'
).properties(width=100, height=150)

In [ ]:
base = alt.Chart().mark_line().encode(
    alt.X('time:T'),
    # alt.Y('rolling_median:Q')
    alt.Y('value:Q')
).properties(width=200, height=150)
(
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20230418'].reset_index()) |\
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20221221'].reset_index()) |\
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20230409'].reset_index())
).resolve_scale(y='shared')

In [ ]:
base = alt.Chart().mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:N')
).properties(width=200, height=150)
sos_dir_data = sos_df_localtime[
    sos_df_localtime.variable.isin(['dir_3m_c', 'dir_20m_c'])
].set_index('time')

(
    alt.layer(base, data=sos_dir_data.loc['20230418'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20221221'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20230409'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20230304'].reset_index()) 
).resolve_scale(y='shared')

# Get mrds for all days (1500-1800) with consistent winds

In [ ]:
import tqdm

In [ ]:
mrds_processed_data = {}

# for d in tqdm.tqdm(days_with_consistent_1500_1800_winds):
for d in tqdm.tqdm([d for d in days_with_consistent_1500_1800_winds if d.month == 4]):
    day_str = str(d).replace('-','')
    mrds_processed_data[day_str] = process_mrds_for_files([
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_21.nc',
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_22.nc',
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_23.nc',
    ])

In [ ]:
len(mrds_processed_data)